In [1]:
!pip install openai faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
import openai
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel

##Cleaned emails

In [3]:
def read_emails_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        emails = file.readlines()
    return emails

def clean_email_data(emails):
    cleaned_emails = []
    current_email = []

    for email in emails:
        # Remove leading/trailing whitespace
        email = email.strip()

        if email.startswith('From'):
            continue

        # When we hit the separator, save the current email content as a single entry
        if email.startswith('-----'):
            if current_email:
                cleaned_emails.append('\n'.join(current_email))  # Join the list into a single string
                current_email = []  # Reset for the next email
        else:
            # Add the current line to the ongoing email content
            current_email.append(email)

    # Add the last email if the file doesn't end with a separator
    if current_email:
        cleaned_emails.append('\n'.join(current_email))

    return cleaned_emails

file_path = '/content/promotional_emails_with_ocr.txt'
emails = read_emails_from_txt(file_path)
cleaned_emails = clean_email_data(emails)

# Print the first cleaned email as an example
print(cleaned_emails[18])



Subject: did you see this?! $3.95 ends today... 🛍
Body: To ensure delivery of Bath & Body Works emails to

your inbox (not bulk or junk folders), please add

bathandbodyworks@e.bathandbodyworks.com to your address book.



If you would like to view this email with images, please click the link below.

https://view.e2.bathandbodyworks.com/?qs=7bc74e5639fe6fedef99fba15d2057ec92ee951b5c839bb7b8fae59fba3b47b62ca93915dddbb65747c4d78e04b96942625ff5a0c4e2472b94efbf205d0daafd78e53eaf21f28bfbd0ce57b04cc6fbdd



Check out what's new at Bath & Body Works. Shop our exclusive

selection of the world's best beauty brands and get access to

special promotions.



>>Visit BathandBodyWorks.com

https://click.e2.bathandbodyworks.com/u/?qs=3abdff6e233b8e25d6d82369ce0b4b18dde49f781259c471acd14aa306b812fa8c169d14ddf18a03af7c4c142aedb8318f4837211b2e2723c651cd6a3e51a9e3



For more information, call 1-800-756-5005 or visit our contact us

page. Please do not reply to this message.



Click the link below to

In [4]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.2 MB/s eta 0:00:00


#Convert content to Vector Embeddings.

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the cleaned emails
email_embeddings = model.encode(cleaned_emails)

# Display sample embeddings (just for verification)
print(email_embeddings.shape)  # (num_emails, embedding_dim)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(153, 384)


In [6]:
import faiss

# Initialize FAISS index (L2 distance search)
embedding_dim = email_embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)

# Add the email embeddings to the index
index.add(np.array(email_embeddings))

# Save the index for later use
faiss.write_index(index, "emails_faiss_index.index")

#Top 3 documents

In [7]:
query = "bath and body works"
query_embedding = model.encode([query])

# Perform the search in FAISS
D, I = index.search(np.array(query_embedding), k=3)  # Retrieve top 3 results

# Show the most relevant emails
for idx in I[0]:
    print(cleaned_emails[idx])


Subject: hey, member: spend $50 on your faves, 🛍 get a Reward! 👏
Body: To ensure delivery of Bath & Body Works emails to

your inbox (not bulk or junk folders), please add

bathandbodyworks@e.bathandbodyworks.com to your address book.



If you would like to view this email with images, please click the link below.

https://view.e2.bathandbodyworks.com/?qs=462db5f6cfdb6fbee9d8943b5b3db6b4ad14e8ab324c210a243e46fab2f93699d03c2867361b26eb51943b956f14927c0969eddd82fe903f1e1d27fe1a2490736104c9a15f39c38829b952bcc780c19a



Check out what's new at Bath & Body Works. Shop our exclusive

selection of the world's best beauty brands and get access to

special promotions.



>>Visit BathandBodyWorks.com

https://click.e2.bathandbodyworks.com/u/?qs=c838cd2be76e7108603cc82b5316c62f039612f72c9e102082302e43a94d5159ef9b9a02048551db47772d2800e121bed5bd7501e8b1e9b134557126f68bbf42



For more information, call 1-800-756-5005 or visit our contact us

page. Please do not reply to this message.



Click th

In [8]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.9 MB/s eta 0:00:00


In [18]:
!pip install python-dotenv

In [19]:
from dotenv import load_dotenv
import os
import cohere

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment.
cohere_api_key = os.getenv('COHERE_API_KEY')

# Initialize the Cohere client with the API key
cohere_client = cohere.Client(cohere_api_key)

##Output using content store.

In [23]:
# Prepare a prompt using retrieved email content
retrieved_email_content = " ".join([cleaned_emails[idx] for idx in I[0]])
product = "Body Mist"

response = cohere_client.generate(
    model='command-xlarge',  # This is a large, powerful model
    prompt=f"Using the following email content and template: {retrieved_email_content}, generate a promotional email for {product}",
    max_tokens=500,
    temperature=0.8
)

# Print the generated text
print(response.generations[0].text)

 Subject: Mist-erious Savings on Body Mist! 💦

Hey, [Member Name]!

We've got some exciting news for you! Our beloved Body Mists are the essence of our brand and we can't wait for you to experience this fragrant adventure. From sugary sweets to tropical tropics, we have everything to satisfy your cravings. And that's not all! We're offering a special promotion on our Body Mist collection for a limited time only. 

So, what are you waiting for? It's time to explore these amazing offers and indulge in some fragrant fun! 

Shop Now: [Insert Button or Link]

Don't forget to add bathandbodyworks@e.bathandbodyworks.com to your address book to ensure our emails land in your primary inbox and you don't miss out on exciting news, exclusive offers, and more! 

Let's mist away with these irresistible Body Mist deals! 

Team B&B

This email was sent to you by BATH & BODY WORKS. To ensure delivery to your inbox (not bulk or junk folders), please add bathandbodyworks@e.bathandbodyworks.com to your a

##Output without content store.

In [14]:
response_2 = cohere_client.generate(
    model='command-xlarge',  # This is a large, powerful model
    prompt=f"Generate a promotional pitch for a Body Mist product from Bath & Body Works.",
    max_tokens=150,
    temperature=0.7
)

# Print the generated text
print(response_2.generations[0].text)

 Want a fragrance that will make you feel fresh and clean without being overpowering? Then you need to check out our Body Mist product from Bath & Body Works! Our lightweight, refreshing mist provides a gentle scent that will linger on your skin for hours. It's perfect for when you're on the go or just need a quick pick-me-up. With a variety of scents to choose from, you're sure to find the perfect fragrance to suit your style. So what are you waiting for? Try our Body Mist today and experience the difference for yourself! Get it today at a special holiday discount only at Bath & Body Works.
